In [ ]:
import random
import numpy as np
import os 
import matplotlib.pyplot as plt 
import cv2 as cv 
os.getcwd()

'/content'

## Dataset Details - Standford's GloVe pre-trained word vectors

GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus. The GloVe pre-trained word vectors dataset contains English word vectors pre-trained on the combined Wikipedia 2014 + Gigaword 5th Edition corpora (6B tokens, 400K vocab). All tokens are in lowercase. This dataset contains 50-dimensional, 100-dimensional and 200-dimensional pre trained word vectors. In this problem we are going to use the 50-dimensional dataset. 

## \# 0. Get an overview on what Glove is
Read up the documentation on glove embeddings, esp. where it gets applied here: https://nlp.stanford.edu/projects/glove/

## Load Dataset
Let's load the dataset first. Each row is indexed as a word vector. Dimension of word vectors is 50. How many words are there in this dataset? Print a few words and see what they are. You don't need to code anything here, just understand the data structure.

In [ ]:
import pandas as pd
import numpy as np
import sys
import csv

csv.field_size_limit(sys.maxsize)
# Load GloVe pre-trained vectors 
# local_file1="./glove/glove.6B.50d.txt" # Make sure this file exists!
filePATH = '/content/drive/MyDrive/Machine_Learning/HW4/glove.6B.50d.txt' 
df = pd.read_csv(filePATH,sep=' ', index_col=0, header=None, engine='python', on_bad_lines = 'skip', quoting = csv.QUOTE_NONE)
print("Dataset shape - Rows: %d, Cols: %d" % (df.shape[0], df.shape[1]))

words = list(df.index)
print("print a few words in the dataset:", words[30:40])

Dataset shape - Rows: 400000, Cols: 50
print a few words in the dataset: ['be', 'has', 'are', 'have', 'but', 'were', 'not', 'this', 'who', 'they']


## \# 1. Print the first few 11 rows of the pandas data frame below

In [ ]:
# Your code HERE - It should execute as expected! 
# (Search for a pandas functionality that can help you do this!)
df.head(11)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
the,0.418000,0.249680,-0.41242,0.121700,0.345270,-0.044457,-0.49688,-0.178620,-0.000660,-0.656600,0.278430,-0.147670,-0.55677,0.146580,-0.00951,0.011658,0.102040,-0.127920,-0.84430,-0.121810,-0.016801,-0.332790,-0.155200,-0.231310,-0.191810,-1.8823,-0.767460,0.099051,-0.421250,-0.195260,4.0071,-0.185940,-0.522870,-0.316810,0.000592,0.007445,0.177780,-0.158970,0.012041,-0.054223,-0.298710,-0.157490,-0.347580,-0.045637,-0.442510,0.187850,0.002785,-0.184110,-0.115140,-0.785810
",",0.013441,0.236820,-0.16899,0.409510,0.638120,0.477090,-0.42852,-0.556410,-0.364000,-0.239380,0.130010,-0.063734,-0.39575,-0.481620,0.23291,0.090201,-0.133240,0.078639,-0.41634,-0.154280,0.100680,0.488910,0.312260,-0.125200,-0.037512,-1.5179,0.126120,-0.024420,-0.042961,-0.283510,3.5416,-0.119560,-0.014533,-0.149900,0.218640,-0.334120,-0.138720,0.318060,0.703580,0.448580,-0.080262,0.630030,0.321110,-0.467650,0.227860,0.360340,-0.378180,-0.566570,0.044691,0.303920
.,0.151640,0.301770,-0.16763,0.176840,0.317190,0.339730,-0.43478,-0.310860,-0.449990,-0.294860,0.166080,0.119630,-0.41328,-0.423530,0.59868,0.288250,-0.115470,-0.041848,-0.67989,-0.250630,0.184720,0.086876,0.465820,0.015035,0.043474,-1.4671,-0.303840,-0.023441,0.305890,-0.217850,3.7460,0.004228,-0.184360,-0.462090,0.098329,-0.119070,0.239190,0.116100,0.417050,0.056763,-0.000064,0.068987,0.087939,-0.102850,-0.139310,0.223140,-0.080803,-0.356520,0.016413,0.102160
of,0.708530,0.570880,-0.47160,0.180480,0.544490,0.726030,0.18157,-0.523930,0.103810,-0.175660,0.078852,-0.362160,-0.11829,-0.833360,0.11917,-0.166050,0.061555,-0.012719,-0.56623,0.013616,0.228510,-0.143960,-0.067549,-0.381570,-0.236980,-1.7037,-0.866920,-0.267040,-0.258900,0.176700,3.8676,-0.161300,-0.132730,-0.688810,0.184440,0.005246,-0.338740,-0.078956,0.241850,0.365760,-0.347270,0.284830,0.075693,-0.062178,-0.389880,0.229020,-0.216170,-0.225620,-0.093918,-0.803750
to,0.680470,-0.039263,0.30186,-0.177920,0.429620,0.032246,-0.41376,0.132280,-0.298470,-0.085253,0.171180,0.224190,-0.10046,-0.436530,0.33418,0.678460,0.057204,-0.344480,-0.42785,-0.432750,0.559630,0.100320,0.186770,-0.268540,0.037334,-2.0932,0.221710,-0.398680,0.209120,-0.557250,3.8826,0.474660,-0.956580,-0.377880,0.208690,-0.327520,0.127510,0.088359,0.163510,-0.216340,-0.094375,0.018324,0.210480,-0.030880,-0.197220,0.082279,-0.094340,-0.073297,-0.064699,-0.260440
and,0.268180,0.143460,-0.27877,0.016257,0.113840,0.699230,-0.51332,-0.473680,-0.330750,-0.138340,0.270200,0.309380,-0.45012,-0.412700,-0.09932,0.038085,0.029749,0.100760,-0.25058,-0.518180,0.345580,0.449220,0.487910,-0.080866,-0.101210,-1.3777,-0.108660,-0.232010,0.012839,-0.465080,3.8463,0.313620,0.136430,-0.522440,0.330200,0.337070,-0.356010,0.324310,0.120410,0.351200,-0.069043,0.368850,0.251680,-0.245170,0.253810,0.136700,-0.311780,-0.632100,-0.250280,-0.380970
in,0.330420,0.249950,-0.60874,0.109230,0.036372,0.151000,-0.55083,-0.074239,-0.092307,-0.328210,0.095980,-0.822690,-0.36717,-0.670090,0.42909,0.016496,-0.235730,0.128640,-1.09530,0.433340,0.570670,-0.103600,0.204220,0.078308,-0.427950,-1.7984,-0.278650,0.119540,-0.126890,0.031744,3.8631,-0.177860,-0.082434,-0.626980,0.264970,-0.057185,-0.073521,0.461030,0.308620,0.124980,-0.486090,-0.008027,0.031184,-0.365760,-0.426990,0.421640,-0.116660,-0.507030,-0.027273,-0.532850
a,0.217050,0.465150,-0.46757,0.100820,1.013500,0.748450,-0.53104,-0.262560,0.168120,0.131820,-0.249090,-0.441850,-0.21739,0.510040,0.13448,-0.431410,-0.031230,0.206740,-0.78138,-0.201480,-0.097401,0.160880,-0.618360,-0.185040,-0.124610,-2.2526,-0.223210,0.504300,0.322570,0.153130,3.9636,-0.713650,-0.670120,0.283880,0.217380,0.144330,0.259260,0.234340,0.427400,-0.444510,0.138130,0.369730,-0.642890,0.024142,-0.039315,-0.260370,0.120170,-0.043782,0.410130,0.179600
"""",0.257690,0.456290,-0.76974,-0.376790

## \# 2. Words Similarity

Similar words have similar embeddings (or vector values). We can use cosine similarity i.e. cos(u,v) = u.v/(|u||v|) to measure vector similarity. u.v is dot product of vectors, |u| is L2 norm of u. Remember, we spoke about computing similarity based on cosine-similarity (as another alternative to correlation) in class?

1. Normalize matrix df by norm of word vectors. 
1. Define a function to find words similarity to a given word.
1. Use the function defined to find the word in examples that is most similar to "happy".

In [ ]:
# Calculate norm of word vectors
vector_norm = np.sqrt(np.square(df).sum(axis=1))

# What would be the dimension of the vector_norm array?
print("length of vector norm:", len(vector_norm), ", Each Vector will contribute towards an element of norm vector")
# Normalize matrix df by norm using .div()
dfn = df.div(vector_norm, axis = 0)
print("dimension of normalised df: ", dfn.shape)

length of vector norm: 400000 , Each Vector will contribute towards an element of norm vector
dimension of normalised df:  (400000, 50)


In [ ]:
# Define a function to find words similar to a given word in a normalized dataframe
def find_word_similarity(word, examples,dataframe):
    """
    Input: word - one string
    examples - List of strings
    dataframe - An indexed normalized dataframe
    """
    ## YOUR CODE HERE
    dotwithexample = dataframe.loc[examples].dot(dataframe.loc[word])
    dotwithexample2 = dotwithexample.sort_values(ascending = False)
    # Calculate dot product of each word in examples to the given word, sorted by value high to low
    # Once you have the sorted values of dot products (notice because of normalization, the dot product is the cosine similarity!),
    # obtain the words corresponding to the sorted values and call it similar_words
    similar_words = dotwithexample2

    # Return words similar to the given word
    return similar_words
    
examples = ["sad", "bad", "evil", "healthy", "ill",
            "beaming", "cheerful", "joyful", "radiant", "glad", "upset",
            "disco", "probably", "hardly", "ephemeral", "close", "cleaning", 
            "maths", "word", "distribution"]

# Use above function to calculate examples' similarity to happy (both "happy" and words in examples are in dfn)
print ( find_word_similarity("happy", examples,dfn) )

0
glad            0.865877
hardly          0.816272
probably        0.747581
bad             0.708395
sad             0.689063
healthy         0.640579
word            0.599150
cheerful        0.575719
upset           0.566424
joyful          0.555032
close           0.554029
ill             0.522978
evil            0.452148
disco           0.324669
beaming         0.289510
cleaning        0.246022
distribution    0.160149
radiant         0.134971
ephemeral       0.132886
maths          -0.011991
dtype: float64


In [ ]:
examples = ["sad", "bad", "evil", "healthy", "ill",
            "beaming", "cheerful", "joyful", "radiant", "glad", "upset",
            "disco", "probably", "hardly", "ephemeral", "close", "cleaning", 
            "maths", "word", "distribution"]

from sklearn.metrics.pairwise import cosine_similarity
similarity2 = pd.DataFrame(cosine_similarity(np.array(df.loc['happy']).reshape(1,50),np.array(df.loc[examples])),columns = examples)
similarity2 = similarity2.T.sort_values(by=[0], ascending=False)
print(similarity2)

                     0
glad          0.865877
hardly        0.816272
probably      0.747581
bad           0.708395
sad           0.689063
healthy       0.640579
word          0.599150
cheerful      0.575719
upset         0.566424
joyful        0.555032
close         0.554029
ill           0.522978
evil          0.452148
disco         0.324669
beaming       0.289510
cleaning      0.246022
distribution  0.160149
radiant       0.134971
ephemeral     0.132886
maths        -0.011991


## \# 3. Goodness of similarity
Comment on the how good the glove embeddings are on finding similar words to a given word using cosine similarity? Glove and word2vec embeddings are based on co-occurence of words in senetences across hundreds of thousands of documents on the web. Would this help explain your observations on word similarity?
What if you replace happy with sad, how do the results change?

**Interpretation:**
Maths got negative similarity with happy. In general, it is often seen that most of the people don't like math or have troubles with math. This similarity seems appropriate. similarly, glad got maximum similarity with happy which also makes sense. The similarity between happy - (hardly and sad) establishes the fact the Glove considers co-occurences in global and well as local corpus of words. sentiments are not factored in this approach of similarity. 

In [ ]:
examples1 = ["happy", "bad", "evil", "healthy", "ill",
            "beaming", "cheerful", "joyful", "radiant", "glad", "upset",
            "disco", "probably", "hardly", "ephemeral", "close", "cleaning", 
            "maths", "word", "distribution"]
print ( find_word_similarity("sad", examples1,dfn) )

0
hardly          0.707888
happy           0.689063
bad             0.664580
glad            0.644321
joyful          0.587249
word            0.559652
probably        0.535477
cheerful        0.531766
upset           0.501137
ill             0.487844
evil            0.401493
healthy         0.372976
disco           0.362746
close           0.311398
radiant         0.244188
ephemeral       0.234799
beaming         0.194717
cleaning        0.135319
maths           0.030129
distribution    0.008951
dtype: float64


similarity between math and sad increased which again justify the co-occurence situation. similarity of disco with sad is greater than with happy, this might be because people tend to go to a disco when they are sad and hence, these words are used more frequently. Cleaning has higher similarity to Happy. 

**Takeaway:** Similarity using GloVe embedding find the words which have similar connotation but doesn't captures the sentiment invloved with the words. 

## \# 4. Correlations
(This question is more of a reflection and building your intuition on how correlations we spoke in class connects to a real-world data set -  Open ended!)
What are some of the most correlated words from the similarity search you did earlier to the word "happy" and "sad". Likewise, what are some of the most uncorrelated words to "happy" and "sad". Does this make sense? How would you improve the results ? If "happy" were a random variable and "sad" was a random variable - What factors make the correlation between "happy" and "sad" (as you computed above) high?

#### Most similar words to "happy" are(>70): 
1. glad
2. hardly 
3. probably 
4. bad

#### Most uncorrelated words to "happy" are(<0.15): 
1. maths
2. ephemeral  
3. radiant  
4. distribution

#### Most similar words to "sad" are(>60): 
1. hardly
2. happy 
3. bad
4. glad

#### Most uncorrelated words to "sad" are(<0.15): 
1. distribution 
2. maths
3. cleaning 

some of these are doesn't make sense as they represents similar category of feeling but they don't necessarily represents same feeling. But vectors in GloVe embedding can be great starting point. 

1. It can also be improved by increasing the dimensinality of the each vector. 
2. Results of glove embedding can be improved by factoring sentiments in the similarity calculation. 

If "Happy" and "sad" where random variables, 2 factors will result in high correaltion: 
1. Probability of co-occurence. They belong to same category "emotions". so, there is high chance of using these words in a particular context. 
2. It is also imperative that, given a sentence happy and sad can be used interchangably to represent two different emotions. example, student was happy with the output of his code. student was sad with the output of his code. all the other words remain same around happy and sad. 




## \# 5. Find nearest neighbourhood

It is helpful to compute the nearest neighbors to a word based on the cosine similarity that we defined earlier, so that given a word we can compute which are the other words which are most similar to it. Sometimes, the nearest neighbors according to this metric reveal overlap of concepts or topics that a word shares. E.g. government might be related to the word politics because they both share topics related to public policy, politicians, parties, elections, etc. The idea is whatever embeddings we are using - word2vec or glove is "hopefully" able to capture these correlations right!

1. Define a function to find the top n similar words to a given word. You can use either dot product of vectors or cosine_simialrity function. Note the search space for words is coming from your pandas data frame (so unlike the similarity problem we worked on earlier, we are not restricted to only a few words to search from - the search space here is the entire vocab captured in your data frame).
1. Find 20 nearest neighborhood for words 'duck' and 'animal'.
1. Find neighborhood intersection of 'duck' and 'animal', to find which words are similar to both 'duck' and 'animal'. This is also related to a similarity measure called "Jaccard Similarity" - Read up on this here: https://en.wikipedia.org/wiki/Jaccard_index

In [ ]:
# define a function to find the top n similar words to a given word in the 'df'

# PART 1
def find_most_similar(df, word, n):
    """
    INPUT: 
    df: Given Data frame
    word: String
    n: Number of similar words to return
    OUTPUT:
    the list of similar words to return
    """
    ## YOUR CODE HERE
    # define and compute the most similar words
    # Use a similarity measure like cosine similarity (like earlier) to do so
    df = df.loc[~df.index.duplicated()]
    words = np.array(df.index)
    similarity = pd.DataFrame(cosine_similarity(np.array(df.loc[word]).reshape(1,50),np.array(df.loc[words])),columns = words)
    similarity = similarity.T.sort_values(by=[0], ascending=False)
    similarity_sorted = similarity.head(n)
    
    return similarity_sorted


# PART 2
# find top 40 similar words to duck
simil1 = find_most_similar(df, "duck", 40)
# find top 40 similar words to animal
simil2 = find_most_similar(df, "animal", 40)

# PART 3
# find the intersection of simil1 and simil2
#intersection =  (concat function of pandas is helpful here)
combined = pd.concat([simil1, simil2], axis=1)
combined.columns = ['duck', 'animal']
combined_final = combined.dropna(how='any')
print (combined_final)

word_labels = ['duck', 'animal'] + list(combined_final.index)
print("\nword labels are:\n", word_labels)


          duck    animal
pig   0.739596  0.735997
cat   0.703251  0.684206
dog   0.693291  0.725226
fish  0.676019  0.728633
meat  0.655953  0.675570
bird  0.643547  0.800324

word labels are:
 ['duck', 'animal', 'pig', 'cat', 'dog', 'fish', 'meat', 'bird']


## \# 6 Word analogies

Suppose you know the word vectors for King, Man and Woman. What is your intuitive answer for the 'riddle', King - Man + Woman = ? 
Let's go through below steps to derive the answer for this 'riddle' using the word embeddings.

1. Use vector arithmetic to define a new vector which equals to k - m + w (e.g. king, man and woman combination).
2. Calculate similarity of all the words in the corpus to the new vector and sort them by their similarity high to low. 
3. Return the top n vectors which have the highest similarity to the new vector.
1. Find the answers for the riddles, 
    1. good:bad::up:?
    1. germany:merkel::america:?

In [ ]:
def solve_riddle(x, y, a, n, dataframe):
    ## YOUR CODE HERE
    # calculate the vector of a + y - x, where a, x, y are in dataframe
    vec_a = np.array(dataframe.loc[a])
    vec_x = np.array(dataframe.loc[x])
    vec_y = np.array(dataframe.loc[y])

    cal_vec = vec_a + vec_y - vec_x
    # calculate distance of words in dataframe to cal_vec, sorted by similarity high to low
    df = dataframe.loc[~dataframe.index.duplicated()]
    df = df.drop([a, x, y])
    words = np.array(df.index)
    similarity = pd.DataFrame(cosine_similarity(np.array(cal_vec).reshape(1,50),
                                                np.array(df.loc[words])),columns = words)
    similarity = similarity.T.sort_values(by=[0], ascending=False)
    similarity_sorted = similarity.head(n)

    # return top n words and distance that closest to cal_vec
    return similarity_sorted 

# Call the solve_riddle function to compute the top answers
print ("King - Man + Woman =\n", solve_riddle("man", "woman", "king", 5,df) )

## YOUR CODE HERE
# Solve the other two riddles
# good:bad::up:?
# # PART 4
print ("\ngood:bad::up:?\n", solve_riddle("good", "bad", "up", 5,df) )

# # germany:merkel::america:?
print ("\ngermany:merkel::america:?\n",solve_riddle("germany", "merkel", "america", 5,df))

King - Man + Woman =
                  0
queen     0.860958
daughter  0.768451
prince    0.764070
throne    0.763497
princess  0.751273

good:bad::up:?
                  0
down      0.850167
falling   0.813844
out       0.792837
dropping  0.782064
off       0.778428

germany:merkel::america:?
                 0
obama    0.694289
barack   0.682594
hillary  0.660910
bush     0.657911
clinton  0.655765
